# Laba

In [9]:
import numpy as np
import pandas as pd
from Classes.PercepTeacher import PerceptronTeacher
from Functions.function_of_activation import sigmoid, dsigmoid
from Functions.errorFunctions import MSE
from Classes.Layer import Layer
from Classes.LayerGen import LayerGenerator
from random import random
from Classes.Perceptron import Perceptron
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


In [10]:
def batch_spliter(x, y, size, num):
    batches = list()
    for _ in range(num):
        js = np.random.shuffle(np.arange(size))
        batches.append((x[js][0], y[js][0]))
    return batches


In [11]:
models = pd.read_csv('../Data/DataLaba2/neo_task_smoted.csv')
X_train, X_test, y_train, y_test = train_test_split(models.drop(
    'hazardous', axis=1), models['hazardous'], test_size=0.1, random_state=42)


In [12]:
X_train.shape


(147592, 4)

In [13]:
layer_mass = LayerGenerator(sizes_of_input_layer=X_train.shape[1],
                            sizes_of_hidden_layers=[
                                X_train.shape[1] // 2, X_train.shape[1] // 4],
                            sizes_of_output_layer=1,
                            summators=[lambda w, x, d: ((w.dot(x.T)).T + d)/(x.shape[0]),
                                       lambda w, x, d: (
                                           (w.dot(x.T)).T + d)/(x.shape[0]),
                                       lambda w, x, d: (
                                           (w.dot(x.T)).T + d)/(x.shape[0]),
                                       lambda w, x, d: ((w.dot(x.T)).T + d)/(x.shape[0])],
                            dsummators=[lambda w, x, d: (x, d),
                                        lambda w, x, d: (x, d),
                                        lambda w, x, d: (x, d),
                                        lambda w, x, d: (x, d)],
                            activation_functions=[sigmoid,
                                                  sigmoid,
                                                  sigmoid,
                                                  sigmoid],
                            dactivation_functions=[dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid],
                            function_of_generation=lambda a, b: np.random.rand(a, b)).generate()
layer_mass


In [14]:
k = []
for i in layer_mass:
    k.append(i.get_weights())
    print(i.get_weights())


[[0.34546116 0.90570528 0.9573566  0.04951128]
 [0.20017312 0.22037932 0.30005758 0.41318394]]
[[0.39309956 0.79168909]]
[[0.5471626]]


In [15]:
teacher = PerceptronTeacher(layers=layer_mass,
                                 step=1e-1,
                                 batch_spliter=lambda a, b: batch_spliter(a, b, a.shape[0]//2, 1),
                                 epochs=1,
                                 error_function=MSE,
                                 derror_function=lambda x, y: 2 * x - 2 * y)


In [16]:
tl = teacher.teach(X_train.to_numpy()[:200], y_train.to_numpy()[:200])


df:  (1, 1) [[0.1636610943853403]]
d_output:  (1, 1) [[-0.41233034580757977]]
a:  (1, 1) [[0.7887146348054855]]
s:  (1, 1) [[1.3481977555553977]]
W:  (1, 1) [[0.547162600053584]]
B:  (1, 1) [[0.9166426052749151]]

res
1)  (1, 1) [[-0.06748243564315431]]
2.1)  (1, 1) [[0.7887146348054855]]
2.2)  (1, 1) [[0.9166426052749151]]
W_Step:  (1, 1) [[-0.005322438458407514]]
df:  (1, 1) [[0.1666438596491351]]
d_output:  (1, 1) [[-0.053224384584075135]]
a:  (1, 2) [[0.5557863433813167, 0.712157662880017]]
s:  (1, 1) [[1.3171949144058388]]
W:  (1, 2) [[0.39309956144831104, 0.7916890868814204]]
B:  (1, 1) [[0.5349080967225958]]

res
1)  (1, 1) [[-0.008869516874540207]]
2.1)  (1, 2) [[0.5557863433813167, 0.712157662880017]]
2.2)  (1, 1) [[0.5349080967225958]]
W_Step:  (1, 2) [[-0.00024647781756297933, -0.0003158247204123714]]
df:  (1, 2) [[0.2468878838921418, 0.20498912608128905]]
d_output:  (1, 2) [[-0.004929556351259586, -0.006316494408247427]]
a:  (1, 4) [[-0.9934050793279926, -0.3821607133467271

ValueError: cannot reshape array of size 4 into shape (1,2)

In [ ]:
for i in tl:
    print(i.get_weights())


[[0.74977195 0.12142957 0.5548425  0.8240878 ]
 [0.36750168 0.64460638 0.54870194 0.54289776]]
[[ 0.32610995  0.63535416]
 [ 0.42572517  0.25316266]
 [ 0.21079298  0.34816474]
 [ 0.11638221  0.17561103]
 [ 0.71255823  0.03200632]
 [ 0.66332394 -0.00992698]
 [ 0.36819017  0.53332385]
 [ 0.36186353  0.75739875]
 [ 0.84165474  0.36949789]
 [ 0.65370243  0.59215472]
 [ 0.01770215 -0.02834116]
 [-0.00999884  0.53288631]
 [ 0.2935368   0.45480814]
 [ 0.85341552  0.86008514]
 [-0.00461621  0.69836967]
 [ 0.53084164  0.03583752]]
[[0.42014684 0.14902651 0.43572448 0.33116788 0.14882357 0.53138518
  0.77361923 0.81551145 0.34744598 0.23068271 0.72875479 0.81226077
  0.67287042 0.14955441 0.64434368 0.63040078]]


In [ ]:
for i, j in zip(k, tl):
    print(i - j.get_weights())


[[-0.06249022 -0.05945566 -0.01389976  0.05980165]
 [-0.06249022 -0.05945566 -0.01389976  0.05980165]]
[[0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]
 [0.03251974 0.03141219]]
[[0.02869293 0.02799562 0.02767675 0.02699963 0.02818478 0.02795249
  0.02854698 0.02908917 0.02935903 0.02942902 0.02623252 0.0275672
  0.02815785 0.03060136 0.02799541 0.02772393]]


In [ ]:
a = Perceptron(tl)
res = [a.predict(i)[0] for i in X_test.to_numpy()]
print(MSE(res, y_test.to_numpy()))
for p, y in zip(res, y_test.to_numpy()):
    print(f'p:{p},\t y:{y},\t d:{abs(p-y)}')


0.271923157262776
p:0.6513022868902407,	 y:0.0,	 d:0.6513022868902407
p:0.6511984960688052,	 y:0.0,	 d:0.6511984960688052
p:0.6513327412303265,	 y:1.0,	 d:0.3486672587696735
p:0.6513015162148271,	 y:0.0,	 d:0.6513015162148271
p:0.6512415452996094,	 y:0.0,	 d:0.6512415452996094
p:0.6513809844897108,	 y:1.0,	 d:0.3486190155102892
p:0.6512912397175761,	 y:1.0,	 d:0.3487087602824239
p:0.6514620671771175,	 y:0.0,	 d:0.6514620671771175
p:0.6514332532961922,	 y:0.0,	 d:0.6514332532961922
p:0.6511928715188315,	 y:0.0,	 d:0.6511928715188315
p:0.6513607872815625,	 y:1.0,	 d:0.34863921271843745
p:0.6512802062475512,	 y:1.0,	 d:0.34871979375244877
p:0.6513586617966619,	 y:0.0,	 d:0.6513586617966619
p:0.6513114413809226,	 y:1.0,	 d:0.34868855861907744
p:0.6512922332808543,	 y:1.0,	 d:0.3487077667191457
p:0.6511886099487512,	 y:0.0,	 d:0.6511886099487512
p:0.6511660636531521,	 y:0.0,	 d:0.6511660636531521
p:0.6513729135877017,	 y:0.0,	 d:0.6513729135877017
p:0.6512566760742325,	 y:1.0,	 d:0.34874332

In [ ]:
np.array(res).mean(), np.array(res).std(), np.array(res).min()


(0.6512869389523382, 0.00010724404225128987, 0.651007750769594)

In [ ]:
np.partition(np.array(res), 5)[5]


0.6510344607833581

In [ ]:
# -1.1027074438226645
r2_score(np.array(res) >= np.array(res).mean()-0e-5, y_test)

-1.0959255568009802